In [1]:
import pickle
from os.path import exists
import warnings
import time
warnings.filterwarnings(action='once')
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
pd.options.display.max_columns = 999

start_time = time.time()

## Setup Stuff (that should come from config files or database later)

CHANNEL: Setting up Channel Transaction Fee (%) for each Channel >> List of Dictionaries

WAREHOUSES: Defining Keys and Shipping Charge Category for Each WareHouse >> List of Dictionaries

get_warehouse_key(warehouse_name): Return Key of the given Warehouse (from the WAREHOUSES) - can be improved

Initializing DEFAULT_TARGET_PROFIT(5%), DEFAULT_SHIP_MARKUP (12%), EXCLUDED_WAREHOUSES, PUNCTUATION_WAREHOUSES (Punctuation has to be retained in these)

PARTS_AUTH_SHIPPING_MODEL: The Machine pretrained Machine Learning Model loaded from pickle
PRICE_FILE_COLUMNS: Columns in the Price File

PRICE_FILE_LOCATION
CatSKU_CHANNELS // Need elaboration


In [2]:
CS_TRANSACTION_FEE = 0.0101
CHANNELS = [
    {'name':'AP Fusion', 'channel_transaction_fee':0.08, 'target_profit': 0.09},

    {'name':'PS Amazon', 'channel_transaction_fee':0.119},
    {'name':'PS Walmart', 'channel_transaction_fee':0.125},
    {'name':'PS Ebay', 'channel_transaction_fee':0.17},  ## Changed from 0.15 on Feb 24 2022
    
    {'name':'BS Amazon', 'channel_transaction_fee':0.12},
    {'name':'BS Walmart', 'channel_transaction_fee':0.12},
    {'name':'BS Ebay', 'channel_transaction_fee':0.12},
    
    {'name':'Mecka', 'channel_transaction_fee': 0.12}
]

WAREHOUSES = [
    # Fully integrated warehouses
    {'key':'C', 'name':'Brock', 'shipping':'free-ish'},
    {'key':'D', 'name':'Dorman Direct', 'shipping':'free-ish', 'target_profit': 0.6},
    {'key':'J', 'name':'PFG', 'shipping':'theirs'},
    {'key':'K', 'name':'Keystone', 'shipping':'theirs'},
    {'key':'N', 'name':'NPW', 'shipping':'ours'},
    {'key':'O', 'name':'Tonsa', 'shipping':'ours'},
    {'key':'P', 'name':'Parts Auth', 'shipping':'theirs'},
    {'key':'Y', 'name':'Motor State', 'shipping':'theirs'},
    # Manual/FTP warehouses
    {'key':'1', 'name':'Jante Wheel', 'shipping':'free'},
    {'key':'2', 'name':'OE Wheels', 'shipping':'theirs'},
    {'key':'6', 'name':'Burco Mirrors', 'shipping':'ours'},
    {'key':'8', 'name':'Race Sport Lighting', 'shipping':'ours', 'target_profit': 0.1},
    {'key':'9', 'name':'Sunbelt APG', 'shipping':'ours'},
    # Low-volume, or unused warehouses
    #{'key':'5', 'name':'KSI Trading'},
    #{'key':'7', 'name':'NTW'},
    #{'key':'H', 'name':'Hanson'},
    #{'key':'3', 'name':'Motor State'},
]

def get_warehouse_key(warehouse_name):
    for warehouse in WAREHOUSES:
        if warehouse['name'] == warehouse_name:
            return warehouse['key']
    return None


DEFAULT_TARGET_PROFIT = 0.05
DEFAULT_SHIP_MARKUP = 1 / 1.12
EXCLUDED_WAREHOUSES = ['A','5','7','H','3', 'O']
PUNCTUATION_WAREHOUSES = ['J','1','C', '9', '8', 'Y']

PARTS_AUTH_SHIPPING_MODEL = 'shipping-research/tree-model.pkl'

PRICE_FILE_COLUMNS = ['CS-SKU-NP', 'MinPrice', 'Shipping', 'Carrier', 'Service', 'Markup',
       'ShipMkup', 'ListMkup', 'PackQty', 'MinQty', 'MaxQty', 'Zip Code',
       'CatSKU', 'OP-Lowest(Y)', 'VND-Lowest(Y)', 'MinMkDown', 'MaxMkUp', 'Interval',
       'BundleSKU', 'Duplicate']

PRICE_FILE_LOCATION = 'price-files'

CatSKU_CHANNELS = ['PS Ebay']

# Main

#### Load in data.

Reading the Price Weight file from Disk

warehouses: DataFrame containing unique warehouse keys

In [3]:
pw = pd.read_csv('inventory/pw-all.csv', low_memory=False, 
                 dtype={'MasterLC':'Int64', 'Zip Code': str})
# pw['MasterLC'] = pw['MasterLC'].astype('Int64')

# Temporarily remove all NPW.
##pw = pw[pw['WD'] != 'N']

warehouses = pw['WD'].unique()

## Top-level processing and filtering.

#### Sad updates.

In the Priceweight data (pw) copying CS-SKU to CS-SKU-NP - basically copying the SKU with punctuation into the non-punctuation SKU column (for Brock 'C')\
Setting MasterLC to 429 if MasterLC is 158 and 429


In [4]:
# Correct the Line Code and CS-SKU-NP for Brock
pw.loc[pw['WD']=='C', 'CS-SKU-NP'] = pw.loc[pw['WD']=='C', 'CS-SKU']
pw.loc[(pw['MasterLC']==158) & (pw['LC']=='429'), 'MasterLC'] = 429

#### Preprocess price file.

Reading MAP (Minimum Advertized Price) from file against each SKU and dropping duplicates if any\
Sorting on 'CS-SKU-NP', dropping duplicates based on CS-SKU-NP and setting CS-SKU-NP as index -- This creates a unique sorted index which has performance benefits https://stackoverflow.com/questions/16626058/what-is-the-performance-impact-of-non-unique-indexes-in-pandas \
Basically, it can search any value in O(1) time\
In the end, we get a Pandas Series with SKU as index and MAP as value

In [5]:
map_prices = pd.read_csv('maps.csv')
map_prices = map_prices.sort_values('CS-SKU-NP').drop_duplicates(subset=['CS-SKU-NP'])
map_prices = map_prices.set_index('CS-SKU-NP')['MAP']

Creating a new column 'item_cost' by copying 'MinPrice'\
Creating filter 'dorman_update_idx'=> where Warehouse is Dorman ('D') and PackQty is not na
Using this filter to convert prices from per pack to per unit

Creating backup copy of cs-sku-np with name cs-sku-np-catsku

For Punctuation warehouses, Create CS-SKU-NP by concatenating Key, MasterLC, | and Part Number\
for non-Puncuation warehouses, remove non-alphanumeric characters from Part Number - This temporary step of copying to temporary variable 'x' can be avoided by filtering and assigning using loc function directly
lambda operator defines a function in a single line - this function is passed as a filter to map

In the Price Weight dataframe (pw), adding a new column 'MAP' (Minimum Advertized Price) with MAP prices from above step if available and 1 if not available in above data (map_prices)\

Removing WeatherTech (310) from Price Weight File

Set not available values indicater (9999) back to actual nan

In [6]:
# Define separate column for managing costs vs MinPrice to avoid confusion.
pw['item_cost'] = pw['MinPrice']

# update Dorman costs (which are per-pack initally) to be per-unit
dorman_update_idx = (pw['WD']=='D') & (pw['PackQty'].notna())
pw.loc[dorman_update_idx, 'item_cost'] = pw.loc[dorman_update_idx, 'item_cost'] / pw.loc[dorman_update_idx, 'PackQty']
del dorman_update_idx

# Make a "backup" copy of cs-sku-np for CatSKU situations
pw['CS-SKU-NP-CatSKU'] = pw['CS-SKU-NP']

#Set CSSKUNP depending on if it's a punctuation warehouse
x = pw[pw['WD'].isin(PUNCTUATION_WAREHOUSES)].copy()
x['CS-SKU-NP'] = x['WD'] + x['MasterLC'].astype(str) + '|' + x['Part Number']
pw.loc[pw['WD'].isin(PUNCTUATION_WAREHOUSES), :] = x

pw = pw.loc[~pw['Part Number'].isnull()] #Removing null Part Numbers

x = pw[~pw['WD'].isin(PUNCTUATION_WAREHOUSES)].copy()
x['CS-SKU-NP'] = (x['WD'] + x['MasterLC'].astype(str) + '|' 
                  + x['Part Number'].map(lambda s: ''.join(filter(str.isalnum, s))))
pw.loc[~pw['WD'].isin(PUNCTUATION_WAREHOUSES), :] = x

pw = pw.join(other=map_prices, on='CS-SKU-NP', how='left')   #optimized code for fetching MAP columnt from map_prices
pw['MAP'].fillna(1, inplace=True)
del map_prices   #This dataset is no longer needed

# Remove WeatherTech (just in case)
pw = pw[pw['MasterLC'] != 310]

# Remove First Stop Brakes Dorman Line
# df = df[~((df['WD']=='D') & df['Part Number'].isin(first_stop_brakes))]
# Nope, actually don't remove them, just set MinQty really high... at the end.

# Remove placeholder values for Weight/ShipWeight
pw.loc[(pw['Weight']==9999), 'Weight'] = np.nan
pw.loc[(pw['ShipWeight']==9999), 'ShipWeight'] = np.nan


pw_cols = pw.columns   #columns in the Price Weight Report

#### Filter parts.

Remove Excluded warehouses from price weight data\
Adjusting invalid PackQty - na => 1\
Considering only values with PackQty <=10 or any Dorman values (we have clean data for Dorman)

In [7]:
# Idea here is to filter out all the lil nasties that we don't want to include.
# This could differ by warehouse, or not.
# Things like, heavy parts, big or oddly shaped parts, 
# parts that are really expensive, or come in packs of many.
# ... See notes on original Jim conversation for what all you should be including here.

# Filter out excluded warehouses.
pw = pw[~pw['WD'].isin(EXCLUDED_WAREHOUSES)]
# Filter out nasty pack quantities. (allow these for Dorman, since we have clean data.)
pw['PackQty'] = pw['PackQty'].fillna(1) # assume PackQty of NA => PackQty=1
pw = pw[(pw['PackQty'] <= 10) | (pw['WD']=='D')]

#### Calculate shipping by warehouse.

Read Shipping Data from file\
create column 'warehouse_key' in this dataframe by fetching key using the 'get_warehouse_key' function
Filter to include entries with Quantity >0 - exclude Quantities=0\
Converting 'Ship Cost' to per item
Filtering records with Shipping Cost > 0.05 and only considering a few columns

Reading price-file-shipping.csv and converting it all to a single dictionary | Using a series with unique index should give us the same performance

#### get_historical_shipping_estimates(df, warehouse_key):
    Filter the main (total) shipping data to the provided warehouse, join/merge df to this filtered shipping data based on SKU creating 'ship_weights'. Filter the records with weights between 0 and 1000 (removing garbage).
    If some clean data is available after this filtering, fit a LinearRegression() model on it to prodict Ship Cost based on dimensions and weights of the items. Displaying the RMS error of this trained model - Train, Test split missing here
    
##### Shipping cost prediction:
    if the sku is available in the recent price-file-shipping, use that shipping cost,
    if their is an order history of the sku, use the mean of the shipping cost of all historical orders
    if the shipping cost (LinearRegression) model exists for the warehouse (i.e. clean data is avaialable), predict the shipping cost using this model
    if their is some historical data available for that warehouse, use its mean
    else use 10 as shipping cost
    
#### calculate_warehouse_shipping(df, warehouse):
Expects a price/ weight dataframe format filtered for the warehouse. Returns with shipping price altered.
Assigns shipping values for each warehouse as per its rules.\
##### pfg
Checks the pfg inventory file to check for shipping and handling costs of the items. Sums these two costs and adds $3.5 to account for taxes. There should be some other way then this loop. (A simple join would probably be better for performance). If the item isn't found in the inventory file, use PFG_DEFAULT_SHIPPING (15)

##### Keystone
KEYSTONE_BASE_SHIPPING=11, KEYSTONE_LTL_SHIPPING=125

##### NPW
Possible overwrite of AC Delco SKUs (ignoring the 35)

In [8]:
shipping_data = pd.read_csv('shipping_data.csv', low_memory=False, parse_dates=['Ship Date'])
shipping_data['warehouse_key'] = shipping_data['Warehouse'].map(get_warehouse_key)
shipping_data = shipping_data[shipping_data['Quantity'] > 0]
shipping_data['Ship Cost'] = shipping_data['Ship Cost'] / shipping_data['Quantity']
shipping_data = shipping_data[shipping_data['Ship Cost'] > 0.05][['CS-SKU','warehouse_key','Ship Cost','Ship Date']]

price_file_shipping = pd.read_csv('price-file-shipping.csv', index_col='CS-SKU-NP')   #Optimized code instead of loop for dictionary conversion

In [9]:
def get_historical_shipping_estimates(df, warehouse_key): # (cssku, warehouse_key):    
    warehouse_shipping_data = shipping_data[(shipping_data['warehouse_key']==warehouse_key)].copy()

    # create warehouse-level shipping model
    feature_cols = ['Weight', 'DimWeight', 'ShipWeight', 'Length', 'Width', 'Height']
    ship_weights = df[['MasterSKU']+feature_cols].merge(warehouse_shipping_data, how='inner',
                                                        left_on='MasterSKU', right_on='CS-SKU').copy().dropna()
    ship_weights = ship_weights[(ship_weights['ShipWeight'] > 0) & (ship_weights['ShipWeight'] < 1000)]  
    if len(ship_weights) > 0:
        model = LinearRegression().fit(ship_weights[feature_cols], ship_weights['Ship Cost'])

         # log model error for audit purposes
        print('RMSE:',mean_squared_error(ship_weights['Ship Cost'], 
                                          model.predict(ship_weights[feature_cols]), squared=False))
    else:
         model = None


    #Getting Shipping price from recent price file
    df['cssku'] = df['WD']+df['MasterSKU']   #Creating cssku to fetch data from recent price file
    df = df.join(other=price_file_shipping, on='cssku', how='left', rsuffix='_pfs')
    df['Shipping'] = df['Shipping_pfs']
    df.drop(columns=['cssku', 'Shipping_pfs'], inplace=True)

    #Getting Mean of Shipping from Historical Shipping Data
    h_ship_avg = warehouse_shipping_data.groupby('CS-SKU').mean()['Ship Cost']
    df = df.join(other=h_ship_avg, how='left', on='MasterSKU')
    df.loc[df['Shipping'].isna(), 'Shipping'] = df.loc[df['Shipping'].isna(), 'Ship Cost']
    df.drop(columns=['Ship Cost'], inplace=True)

    #Getting Shipping Prediction from Dimensional LinearRegression model
    if len(ship_weights) > 0:
        df['ship_model'] = model.predict(df[feature_cols].fillna(0))
        df.loc[df['Shipping'].isna(), 'Shipping'] = df.loc[df['Shipping'].isna(), 'ship_model']
        df.drop(columns=['ship_model'], inplace=True)

    
    #Getting Warehouse Shipping Mean
    h_warehouse_avg = warehouse_shipping_data.groupby('warehouse_key').mean()['Ship Cost']
    df = df.join(other=h_warehouse_avg, on='WD')
    df.loc[df['Shipping'].isna(), 'Shipping'] = df.loc[df['Shipping'].isna(), 'Ship Cost']
    df.drop(columns=['Ship Cost'], inplace=True)

    #Baseline value of 10
    df.loc[df['Shipping'].isna(), 'Shipping'] = 10
    
    return df['Shipping'].values

# Expects something formatted like a price/weight DF, filtered for a warehouse
# returns the price/weight DF with shipping altered
def calculate_warehouse_shipping(df, warehouse):

    df = df.copy()
    print(warehouse)
    if warehouse=='D': # Dorman        
        df.loc[(df['item_cost'] <= 30), ['Shipping', 'ShipMkup']] = 6, DEFAULT_SHIP_MARKUP # flat rate
        df.loc[(df['item_cost'] > 30), ['Shipping', 'ShipMkup']] = 0, 1
        #df['ShipMkup'] = 1
        
    elif warehouse=='C': # Brock
        df.loc[(df['item_cost'] <= 50), 'Shipping'] = 12 # estimate / avg.
        df.loc[(df['item_cost'] > 50), 'Shipping'] = 0
        df['ShipMkup'] = 1 / 1.1 # to account for returns not being accepted
    elif warehouse=='P': # Parts Auth
        df['Shipping'] = get_historical_shipping_estimates(df, warehouse)
        df['ShipMkup'] = DEFAULT_SHIP_MARKUP
        '''
        with open(PARTS_AUTH_SHIPPING_MODEL, 'rb') as f:
            m = pickle.load(f)
        df['lwh'] = df['Length'] * df['Width'] * df['Height']
        df['Shipping'] = m.predict(df[['Weight','Length','Width','Height','lwh']].fillna(0))
        df['ShipMkup'] = DEFAULT_SHIP_MARKUP
        '''
    elif warehouse=='1': # Jante
        df['Shipping'] = 0
        df['ShipMkup'] = 1
    elif warehouse=='J': # PFG        
        #Reading Shipping and Handling Cost from PGF inventory file and using their sum as Shipping Cost
        PFG_DEFAULT_SHIPPING = 15
        inv = pd.read_csv('inventory/pfg.txt', sep='\t', encoding_errors='ignore', escapechar='\\', low_memory=False, usecols=['SKU','SHIPPING_COST','HANDLING_COST'], index_col='SKU' )
        inv['pfg_cost'] = inv[['SHIPPING_COST', 'HANDLING_COST']].sum(1)
        inv.drop(columns=['SHIPPING_COST','HANDLING_COST'], inplace=True)
        df = df.join(other=inv, on='Part Number', how='left')
        df.loc[df['WD']=='J', 'pfg_cost'].fillna(PFG_DEFAULT_SHIPPING, inplace=True)
        df.loc[df['WD']=='J', 'Shipping'] = df.loc[df['WD']=='J', 'pfg_cost']
        df.drop(columns='pfg_cost', inplace=True)

        # ADD AN EXTRA $3.50 TO ACCOUNT FOR TAX MESS
        df['Shipping'] = df['Shipping'] + 3.50 
        
        df['ShipMkup'] = DEFAULT_SHIP_MARKUP
    elif warehouse=='K': # Keystone
        KEYSTONE_BASE_SHIPPING = 11.0
        KEYSTONE_LTL_SHIPPING = 125.0
        inv = pd.read_csv('inventory/keystone.csv', low_memory=False)
        # Since inventory file open, manage duplicate part # issue in Keystone by matching with UPC
        inv['PartNumber'] = inv['PartNumber'].str.replace('=','').str.replace('"','')
        ##inv['KeystoneShipping'] = (inv['UPS_Ground_Assessorial'] + KEYSTONE_BASE_SHIPPING).fillna(0)
        inv['KeystoneShipping'] = (KEYSTONE_BASE_SHIPPING)
        inv.loc[inv['UPSable']==False, 'KeystoneShipping'] = KEYSTONE_LTL_SHIPPING
        inv = inv.sort_values('KeystoneShipping', ascending=False).drop_duplicates(subset=['VendorCode','PartNumber'])
        df = df.merge(inv[['VendorCode','PartNumber','KeystoneShipping']], 
                      how='left', left_on=['LC','Part Number'], right_on=['VendorCode','PartNumber'])
        df['Shipping'] = df['KeystoneShipping']
        df['ShipMkup'] = 1

    elif warehouse=='6': # Burco Mirrors
        #df['Shipping'] = get_historical_shipping_estimates(df, warehouse)
        #df['ShipMkup'] = DEFAULT_SHIP_MARKUP
        df['Shipping'] = 8 # estimate / avg
        df['ShipMkup'] = 1
    elif warehouse=='A': # APW
        df['Shipping'] = get_historical_shipping_estimates(df, warehouse)
        df['ShipMkup'] = DEFAULT_SHIP_MARKUP
    elif warehouse=='2': # OE Wheels
        df['Shipping'] = get_historical_shipping_estimates(df, warehouse)
        df['ShipMkup'] = DEFAULT_SHIP_MARKUP
    elif warehouse=='5': # KSI Trading
        df['Shipping'] = get_historical_shipping_estimates(df, warehouse)
        df['ShipMkup'] = DEFAULT_SHIP_MARKUP
    
    elif warehouse=='7': # NTW
        df['Shipping'] = get_historical_shipping_estimates(df, warehouse)
        df['ShipMkup'] = DEFAULT_SHIP_MARKUP
    elif warehouse=='8': # Race Sport Lighting
        ##df['Shipping'] = df['Weight'].map(lambda w: 15 if (pd.isna(w) or w >= 1) else 6)
        ##df['ShipMkup'] = DEFAULT_SHIP_MARKUP        
        df['Shipping'] = 18
        df['ShipMkup'] = 1        
    elif warehouse=='9': # Sunbelt APG
        df['Shipping'] = get_historical_shipping_estimates(df, warehouse)
        df['ShipMkup'] = DEFAULT_SHIP_MARKUP
    
    elif warehouse=='N': # NPW
        ##df['Shipping'] = get_historical_shipping_estimates(df, warehouse)
        df.loc[df['LC']=='ACD', 'Shipping'] = 35 # set AC Delco skus to $35 shipping
        df['Shipping'] = 12
        df['ShipMkup'] = DEFAULT_SHIP_MARKUP
    
    elif warehouse=='O': # Tonsa
        df['Shipping'] = get_historical_shipping_estimates(df, warehouse)
        df['ShipMkup'] = DEFAULT_SHIP_MARKUP
        
    elif warehouse=='Y': # MotorState
        df.loc[(df['item_cost'] <= 39), 'Shipping'] = 13
        df.loc[((df['item_cost'] > 39) & (df['item_cost'] <= 99.99)), 'Shipping'] = 11.5
        df.loc[(df['item_cost'] >= 100), 'Shipping'] = 10
        df['ShipMkup'] = 1        
        
    else:
        pass
    df['ShipMkup'] = df['ShipMkup'].fillna(1)
    return df[['Shipping','ShipMkup']]

In [10]:
warnings.filterwarnings(action='ignore')

In [11]:
dfs = []
for warehouse in pw['WD'].unique().tolist():
    wdf = pw[pw['WD']==warehouse].copy()
    wdf.loc[:, ['Shipping','ShipMkup']] = calculate_warehouse_shipping(wdf.loc[wdf['WD']==warehouse, :], 
                                                                           warehouse).values
    print('Proportion of parts missing shipping:', wdf['Shipping'].isna().mean())
    dfs.append(wdf)
pw = pd.concat(dfs, ignore_index=True)
pw['ShipMkup'] = pw['ShipMkup'].round(3)

C
Proportion of parts missing shipping: 0.0
D
Proportion of parts missing shipping: 0.0
1
Proportion of parts missing shipping: 0.0
2
Proportion of parts missing shipping: 0.0
4
Proportion of parts missing shipping: 1.0
6
Proportion of parts missing shipping: 0.0
8
Proportion of parts missing shipping: 0.0
9
Proportion of parts missing shipping: 0.0
K
Proportion of parts missing shipping: 0.0
Y
Proportion of parts missing shipping: 0.0
N
Proportion of parts missing shipping: 0.0
P
RMSE: 5.615580604223316
Proportion of parts missing shipping: 0.0
J
Proportion of parts missing shipping: 0.0


In [12]:
#Deleting datasets from memory which are no longer needed
del shipping_data, price_file_shipping

#### Set inventory constraints

In [13]:
pw.loc[:, ['MinQty','MaxQty']] = 2, 12   ## Changed Min to two from three

#### Set price file defaults.

In [14]:
pw['ListMkup'] = .65
pw['SourceQty'] = None
pw['Source'] = None
pw['BundleSK'] = None
pw['Carrier'] = 'FedEx'
pw['Service'] = 'GroundHD'

Set OP-Lowest(Y) and VND-Lowest(Y) to "N" for Dorman

In [15]:
pw.loc[(pw['CS-SKU-NP'].str[0]=='D'), ['OP-Lowest(Y)','VND-Lowest(Y)'] ] = "N", "N"

## Quick fix for shipping

Where shipping is current zero for Tonsa, set it to 20. And 15 for Sunbelt.

In [16]:
pw.loc[((pw['Shipping'] == 0) & (pw['CS-SKU-NP'].str[0] == 'O')), 'Shipping'] = 20
pw.loc[((pw['Shipping'] == 0) & (pw['CS-SKU-NP'].str[0] == '9')), 'Shipping'] = 15

1.5x ~Tonsa, Sunbelt~, and Parts Auth shipping.

In [17]:
# pw.loc[(pw['CS-SKU-NP'].str[0].isin(['O','9','P'])), 'Shipping'] *= 1.5
pw.loc[(pw['CS-SKU-NP'].str[0]=='P'), 'Shipping'] *= 1.5

Double Eagle Eye Shipping.

In [18]:
pw.loc[(pw['CS-SKU-NP'].str[:4]=='P754'), 'Shipping'] *= 2.0

Bumper is expensive to ship.

In [19]:
pw.loc[pw['CS-SKU-NP']=='429|6448-0006', 'Shipping'] = 30

Another expensive shipping update from order: PSA669874628

In [20]:
pw.loc[pw['CS-SKU-NP']=='551|S6585B', 'Shipping'] = 46

Expensive shipping for part from 12/20/2021

In [21]:
pw.loc[pw['CS-SKU-NP']=='P576|3292', 'Shipping'] = 90

Expensive shipping for part from 12/27/2021

In [22]:
pw.loc[pw['CS-SKU-NP']=='P550|290073', 'Shipping'] = 100

In [23]:
pw.loc[pw['CS-SKU-NP']=='P308|55621', 'Shipping'] = 161

In [24]:
pw.loc[pw['CS-SKU-NP']=='P557|277504', 'Shipping'] = 35

In [25]:
pw.loc[pw['CS-SKU-NP']=='P643|ESK5752', 'Shipping'] = 34

In [26]:
pw.loc[pw['CS-SKU-NP']=='P551|40722A', 'Shipping'] = 7

In [27]:
pw.loc[pw['CS-SKU-NP']=='N643|AR8265XPR', 'PackQty'] = 1

In [28]:
pw.loc[pw['CS-SKU-NP']=='P123|33660', 'PackQty'] = 1

PA Shipping Costs from Umer analysis:

In [29]:
pa_shipping_data = pd.read_csv('PA Shipping Costs.csv', low_memory=False)
pa_shipping_data['WD'] = 'P'

pa_shipping_data = pa_shipping_data.sort_values(by='Row Labels', ascending=False)
pa_shipping_data.drop_duplicates(subset='Row Labels', keep="first")

pw = pw.merge(pa_shipping_data, how='left', left_on=['MasterSKU', 'WD'], right_on=['Row Labels', 'WD'])
pw.loc[pw['Final Shipping Cost'] > 0, 'Shipping'] = pw['Final Shipping Cost']
pw = pw[pw_cols]   #Restoring columns before this Merge

del pa_shipping_data   #removing variables no longer needed

NPW Pack Corrections and updates from Abdullah

In [30]:
pw.loc[pw['CS-SKU-NP']=='N223|97469', 'PackQty'] = 10
pw.loc[pw['CS-SKU-NP']=='N178|VL1093', 'PackQty'] = 4
pw.loc[pw['CS-SKU-NP']=='N223|98288', 'PackQty'] = 10 #PackQty issue - 20-May-2022

In [31]:
pw['MasterSKUn'] = pw['CS-SKU-NP'].str[1:]
shipping_fix = pd.read_csv('Shipping Corrections.csv', low_memory=False, encoding='unicode_escape')

pw = pw.merge(shipping_fix, how='left', left_on=['MasterSKUn', 'WD'], right_on=['SKU', 'WD'])
pw.loc[pw['Final Shipping Cost'] > 0, 'Shipping'] = pw['Final Shipping Cost']
pw = pw[pw_cols]

del shipping_fix   #removing variables no longer needed

In [32]:
new_shipping = pd.read_csv('newshipcosts.csv', low_memory=False)

pw = pw.merge(new_shipping, how='left', left_on=['MasterSKU'], right_on=['CS_SKU'])
pw.loc[pw['NewShipCost'] > 0, 'Shipping'] = pw['NewShipCost']
pw = pw[pw_cols]

del new_shipping   #removing variables no longer needed

Set MinQty really high for First Stop Brakes (Dorman line) to avoid actually selling any.

In [33]:
first_stop_brakes = pd.read_excel('8-2 Change 56 Brake Dropship and Stocking.xlsx', 
                                  skiprows=2, sheet_name='Dropship Price').rename(columns={'MATERIAL':'pn'})['pn']
pw.loc[(pw['WD']=='D') & pw['Part Number'].isin(first_stop_brakes), ['MinQty','MaxQty']] = 100, 100

del first_stop_brakes   #removing variables no longer needed

Handle RSL skus for MAP > Calculated Cost

In [34]:
#Reading RSL Inventory and Removing Duplicate SKUs
rsl_inventory = pd.read_csv('inventory/rsl.csv', low_memory=False)
rsl_inventory.sort_values(by='MAP', ascending=False, inplace=True)
rsl_inventory.drop_duplicates(subset=['SKU'], inplace=True, keep='first')
rsl_inventory['CS-SKU-NP'] = '8329|' + rsl_inventory['SKU']
rsl_inventory.set_index('CS-SKU-NP', inplace=True)


pw = pw.join(other = rsl_inventory, on='CS-SKU-NP', rsuffix='_rsl')
pw['tmp_mkup'] = pw['item_cost'] / ( (1 + 0.05) * (pw['item_cost'] + pw['Shipping']) / (1 - 0.15) - pw['Shipping'])
pw['tmp_price'] = pw['item_cost']/pw['tmp_mkup'] + pw['Shipping']/pw['ShipMkup']

pw.loc[pw['tmp_price']<pw['MAP_rsl'], ['MinPrice', 'item_cost']] = pw.loc[pw['tmp_price']<pw['MAP_rsl'], 'MAP_rsl']
pw.loc[pw['tmp_price']<pw['MAP_rsl'], ['Shipping', 'ShipMkup']] = 0

pw = pw[pw_cols]   #Restoring columns
del rsl_inventory   #removing variables no longer needed

In [35]:
motorstate = pd.read_csv('inventory/motorstate.csv', low_memory=False)
#motorstate['PartNumber'] = motorstate.PartNumber.str[3:]
motorstate['PartNumber'] = 'Y' + motorstate['PartNumber']
#pw = pw.merge(motorstate, how='left', left_on=['Part Number', 'WD'], right_on=['PartNumber', 'WD'])

pw['tkey'] = pw['WD'] + pw['LC'] + pw['Part Number']

pw = pw.merge(motorstate, how='left', left_on=['tkey'], right_on=['PartNumber'])
pw.drop(columns='tkey', inplace = True)

del motorstate   #removing variables no longer needed

In [36]:
pw.drop(pw.loc[pw['AirRestricted'] == 'YES'].index, inplace=True)
pw.drop(pw.loc[pw['TruckFrtOnly'] == 'YES'].index, inplace=True)

In [37]:
pw.drop(pw.loc[(pw['WD'] == 'Y') & (pw['MasterLC'] != 261)].index, inplace=True)
pw = pw [pw_cols]   #Restoring Columns

NPW Min Order Qty

In [38]:
'''
npw_packqty = pd.read_csv('NPW PackQty.csv', low_memory=False)
npw_packqty['WD'] = 'N'
npw_packqty.set_index(['WD', 'Line Code', 'Part Number'], inplace=True)
npw_packqty

pw1 = pw.join(other=npw_packqty, on=['WD', 'LC', 'Part Number'], how='left')
pw1.loc[~pw1['NPW_PackQty'].isna()]

f = (pw1['WD']=='N') & (~pw1['NPW_PackQty'].isna())   #Creating the filter for items to be updated
pw1.loc[f & (pw1['PackQty']!=pw1['NPW_PackQty'])].to_excel('NPW PackQty Not Matching.xlsx')
'''

"\nnpw_packqty = pd.read_csv('NPW PackQty.csv', low_memory=False)\nnpw_packqty['WD'] = 'N'\nnpw_packqty.set_index(['WD', 'Line Code', 'Part Number'], inplace=True)\nnpw_packqty\n\npw1 = pw.join(other=npw_packqty, on=['WD', 'LC', 'Part Number'], how='left')\npw1.loc[~pw1['NPW_PackQty'].isna()]\n\nf = (pw1['WD']=='N') & (~pw1['NPW_PackQty'].isna())   #Creating the filter for items to be updated\npw1.loc[f & (pw1['PackQty']!=pw1['NPW_PackQty'])].to_excel('NPW PackQty Not Matching.xlsx')\n"

OE wheels price MAP fix 11/09

In [39]:
##oe_wheel_update = pd.read_excel('OE wheel price increase11-9.xlsx', sheet_name='Sheet1')
oe_wheel_update = pd.read_excel('OE Wheel Shipping.xlsx', sheet_name='Sheet1', dtype={'UPC':str})
oe_wheel_update.drop_duplicates(subset=['UPC'], inplace=True, keep='last')

oe_wheel_update['CS-SKU-NP'] = '2387|' + oe_wheel_update['UPC']
oe_wheel_update.set_index('CS-SKU-NP', inplace=True)

pw = pw.join(other = oe_wheel_update, on='CS-SKU-NP', rsuffix='_oe')
pw.loc[~pw['Shipping (Est)'].isna(), 'Shipping'] = pw.loc[~pw['Shipping (Est)'].isna(), 'Shipping (Est)']
pw = pw[pw_cols]   #Restoring Columns
del oe_wheel_update   #removing variables no longer needed

PA packQty Correction from inventory

In [40]:
pa_inv = pd.read_csv('inventory/pa.csv', low_memory=False, encoding= 'unicode_escape')
pa_inv['Part'] = pa_inv['Part'].str.replace('-', '', regex=False )
pa_inv['Part'] = pa_inv['Part'].str.replace('.', '', regex=False )
pa_inv['Part'] = pa_inv['Part'].str.replace('/', '', regex=False )
pa_inv['Total_Stock'] = pa_inv[['BxStock', 'ByStock', 'NYStock', 'DCStock', 'AZStock', 'CAStock', 'GAStock']].sum(axis=1)


pw = pw.merge(pa_inv, how='left', left_on=['Part Number', 'LC'], right_on=['Part', 'Line'], suffixes=('', 'y'))
pw.loc[~pw['Packs'].isna(), 'PackQty'] = pw.loc[~pw['Packs'].isna(), 'Packs']

#Manual Fetching the Price and Quantity from PA inventory
#pw.loc[~pw['Price'].isna(), 'MinPrice'] = pw.loc[~pw['Price'].isna(), 'Price']
#pw.loc[~pw['Price'].isna(), 'item_cost'] = pw.loc[~pw['Price'].isna(), 'Price']
#pw.loc[~pw['Total_Stock'].isna(), 'Qty'] = pw.loc[~pw['Total_Stock'].isna(), 'Total_Stock']

pw = pw[pw_cols]   #Restoring Columns

del pa_inv   #removing variables no longer needed

Route Dorman Drive Shafts to Dorman only (remove it from any other warehouse)

In [41]:
DDS = pd.read_csv('Dorman Drive Shafts.csv', low_memory=False)   #Loading the Dorman DriveShafts List
pw = pw.merge(right = DDS, how='left', on='MasterSKU', indicator=True)   #Merging the Dorman DriveShafts to the Price Weight Report
pw.drop(pw.query("_merge == 'both' and WD!='D'").index, inplace=True)   #Dropping the DormanDriveShafts from any other Warehouse Except Dorman
pw.drop(columns = '_merge', inplace=True)   #Drop the merging indicator column
del DDS   #Delete the Dorman DriveShafts list as it is no longer needed

Force shipping cost and ship markup for RSL Skus

In [42]:
pw.loc[(pw['WD'] == '8') & (pw['Shipping'] == 0), ['Shipping', 'ShipMkup']] = 18, 1

#### Calculate markups and format/write price files, by channel.

Load noncat SKUs from file

In [43]:
noncats = pd.read_csv('non-cat-skus.csv', skiprows=1, low_memory=False)[['CS Linecode','Part Number']]
noncats['cssku'] = noncats['CS Linecode'] + '|' + noncats['Part Number']
noncat_skus = noncats['cssku'].dropna()

In [44]:
def process_channel(channel):    
    pf = pw.copy()
    
    
    #Excluding brands due to eBay Violation
    if 'Ebay' in channel['name']:
        #Excluding AFE
        pf = pf.loc[pf['MasterLC']!=510]

        #Excluding Jet Chips
        pf = pf.loc[pf['MasterLC']!=161]

        #Excluding Russel
        pf = pf.loc[pf['MasterLC']!=265]
        
        #Excluding Diablo
        pf = pf.loc[pf['MasterLC']!=590]
        
        #Excluding SCT Performance
        pf = pf.loc[pf['MasterLC']!=837]
        
        #Removing the Holley Carburetor Repair Kit and Magnaflow Exhause System Kits
        eBay_Violation_skus = pd.read_csv('EBay Violation SKUs.csv', low_memory = False)        
        pf = pf.merge(eBay_Violation_skus, how='left', left_on='MasterSKU', right_on='SKU')
        pf = pf.loc[pf['SKU'].isnull()]
        pf.drop(columns = 'SKU', inplace = True)

        

    # Filter price file if APF or Mecka
    if channel['name'] == 'AP Fusion':
        apf_csskus = pd.read_csv('apf-skus.csv')['cssku']
        apf_csskus = apf_csskus.drop_duplicates()
        pf = pf[pf['CS-SKU-NP'].str[1:].isin(apf_csskus)]
    if channel['name'] == 'Mecka':
        pf = pf[(pf['WD']=='D') & (pf['item_cost'] >= 30)]
    # FIlter BS Walmart 
    if channel['name'] == 'BS Walmart':
        pf.drop(pf.loc[pf['WD'] == 'Y'].index, inplace=True)
        ##pf.drop(pf.loc[pf['WD'] == 'P'].index, inplace=True)
        pf.drop(pf.loc[pf['WD'] == 'J'].index, inplace=True)
        pf.drop(pf.loc[pf['WD'] == 'C'].index, inplace=True)
        pf.drop(pf.loc[pf['WD'] == '8'].index, inplace=True)        


    # # Calculate markups
    target_profit = channel['target_profit'] if 'target_profit' in channel else DEFAULT_TARGET_PROFIT
    channel_fees = channel['channel_transaction_fee'] + CS_TRANSACTION_FEE

    pf['our_cost'] = np.nan
    pf['our_markup'] = np.nan

    bfilter = pf['WD']=='C'
    pf.loc[bfilter, 'our_cost'] = pf.loc[bfilter, 'item_cost']

    # Handling Brock
    #+ $6 for any item costing < $50              
    pf.loc[bfilter & (pf['item_cost']<=50), 'our_cost' ] = pf.loc[bfilter & (pf['item_cost']<=50), 'item_cost' ] + 4
    # # 10% marup for all items, as buffer against no-return policy
    pf.loc[bfilter, 'our_cost' ] = pf.loc[bfilter, 'our_cost' ] * 1.1
    # calculate markup using same formula but with our updated cost value    
    pf.loc[bfilter, 'our_markup'] = pf.loc[bfilter, 'our_cost'] / (( 1+target_profit) * (pf.loc[bfilter, 'our_cost'] + pf.loc[bfilter, 'Shipping'])/ (1-channel_fees) - pf.loc[bfilter, 'Shipping'])
    # then adjust to find what the equivalent markup would be (to get to same final price) with the original cost value
    pf.loc[bfilter, 'Markup'] = pf.loc[bfilter, 'item_cost'] * pf.loc[bfilter, 'our_markup'] / pf.loc[bfilter, 'our_cost']
    pf.drop(columns=['our_cost', 'our_markup'], inplace=True)

    # handle Jante, certain skus specially
    if ( channel['name'] == 'PS Ebay'):
        f_1367 = pf['CS-SKU-NP'].str[:4]=='1367'
        pf.loc[f_1367, 'Markup'] = pf.loc[f_1367, 'item_cost'] / ( (1 + target_profit) * (pf.loc[f_1367, 'item_cost'] + pf.loc[f_1367, 'Shipping']) / (1 - channel_fees) - pf.loc[f_1367, 'Shipping'])

        # Added 1% more to channel fees  ---------------- this will never execute due to logical error - it will only execute if above condition is false which will never happen because 367 is the only line Jante has
        f_jante_n1367 = (pf['CS-SKU-NP'].str[:4]!='1367') & (pf['CS-SKU-NP'].str[0]=='1')
        pf.loc[f_jante_n1367, 'Markup'] = pf.loc[f_jante_n1367, 'item_cost'] / ( (1 + target_profit) * (pf.loc[f_jante_n1367, 'item_cost'] + pf.loc[f_jante_n1367, 'Shipping']) / (1 - (channel_fees + 0.01)) - pf.loc[f_jante_n1367, 'Shipping'])


    # Dorman increase profit
    f_dorman = pf['CS-SKU-NP'].str[0]=='D'   #Defining Filter for Dorman
    
    pf.loc[f_dorman, 'Markup']  = pf.loc[f_dorman, 'item_cost'] / ( (1 + 0.06) * (pf.loc[f_dorman, 'item_cost'] + pf.loc[f_dorman, 'Shipping']) / (1 - channel_fees) - pf.loc[f_dorman, 'Shipping'])
    g_300 = pf['item_cost'] >= 300
    pf.loc[f_dorman & g_300, 'Markup'] = pf.loc[f_dorman & g_300, 'item_cost'] / ( (1 + 0.0575) * (pf.loc[f_dorman & g_300, 'item_cost'] + pf.loc[f_dorman & g_300, 'Shipping']) / (1 - channel_fees) - pf.loc[f_dorman & g_300, 'Shipping'])
    g_500 = pf['item_cost'] >= 500
    pf.loc[f_dorman & g_500, 'Markup'] = pf.loc[f_dorman & g_500, 'item_cost'] / ( (1 + 0.055) * (pf.loc[f_dorman & g_500, 'item_cost'] + pf.loc[f_dorman & g_500, 'Shipping']) / (1 - channel_fees) - pf.loc[f_dorman & g_500, 'Shipping'])
    g_1000 = pf['item_cost'] >= 1000
    pf.loc[f_dorman & g_1000, 'Markup'] = pf.loc[f_dorman & g_1000, 'item_cost'] / ( (1 + 0.05) * (pf.loc[f_dorman & g_1000, 'item_cost'] + pf.loc[f_dorman & g_1000, 'Shipping']) / (1 - channel_fees) - pf.loc[f_dorman & g_1000, 'Shipping'])
    
    # base case, all other skus
    f_rem = pf['Markup'].isna()   #Defining Filter for remaining Data
    
    if channel['name'] == 'AP Fusion':
        pf.loc[f_rem, 'ShipMkup'] = 0.95
        pf['Shipmarkedup'] = pf['Shipping'] / pf['ShipMkup']
        pf.loc[f_rem, 'Markup'] = pf.loc[f_rem, 'item_cost'] / ( (1 + target_profit) * (pf.loc[f_rem, 'item_cost'] ) / (1 - channel_fees) + (pf.loc[f_rem, 'Shipmarkedup']/(1 - channel_fees) - pf.loc[f_rem, 'Shipmarkedup']))        
        pf.drop (columns='Shipmarkedup', inplace=True)
    else:
        pf.loc[f_rem, 'Markup'] = pf.loc[f_rem, 'item_cost'] / ( (1 + target_profit) * (pf.loc[f_rem, 'item_cost'] + pf.loc[f_rem, 'Shipping']) / (1 - channel_fees) - pf.loc[f_rem, 'Shipping'])        
        
        
    # Format price file.
    if channel['name'] in CatSKU_CHANNELS:
        pf['CatSKU'] = (~pf['CS-SKU-NP'].str[1:].isin(noncat_skus)).map(lambda x: 'Y' if x else 'N')
        pf.loc[pf['CatSKU']=='Y', 'CS-SKU-NP'] = pf.loc[pf['CatSKU']=='Y', 'CS-SKU-NP-CatSKU']
    else:
        pf['CatSKU'] = 'N'
        

    # Clean up
    pf.loc[pf['Shipping']==0, 'ShipMkup'] = 1
    # pf.loc[pf['MinPrice'] < 1,'MinPrice'] = 1
    # pf.loc[pf['Markup'] < .1,'Markup'] = .1
    # pf.loc[pf['Markup'] > 1,'Markup'] = 1
    # pf['Markup'] = pf['Markup'].round(3)
    # pf['Shipping'] = pf['Shipping'].round(2)
    # pf.loc[pf['Shipping'].lt(0), 'Shipping'] = 0    
    pf['total_cost'] = pf['item_cost'] + pf['Shipping'] # hoping this will fix most examples of Dorman going thru PA    
    #pf['total_cost'] = (pf['item_cost'] * pf['PackQty']) / pf['Markup'] + pf['Shipping']/pf['ShipMkup']   #Computing the Final Total cost - For Future Use
    

    # give dorman direct preference over dorman skus from other warehouses
    dorman_prefs = pf[(pf['WD']=='D') & (pf['Qty'] > 0)]['MasterSKU'].copy().tolist()
    pf = pf[~((pf['WD'] != 'D') & (pf['MasterSKU'].isin(dorman_prefs)))]

    
    pf = pf.sort_values(['MasterSKU','Qty','total_cost'], ascending=[True,False,True])\
           .drop_duplicates(subset=['MasterSKU'], keep='first')
    
    #Alternate
    #pf = pf.sort_values(['MasterSKU','total_cost', 'Qty'], ascending=[True,True,False])\
    #        .drop_duplicates(subset=['MasterSKU'], keep='first')


    # Return the Price File
    return pf

In [45]:
def process_channel_new(channel):    
    pf = pw.copy()
    
    
    #Excluding brands due to eBay Violation
    if 'Ebay' in channel['name']:
        #Excluding AFE
        pf = pf.loc[pf['MasterLC']!=510]

        #Excluding Jet Chips
        pf = pf.loc[pf['MasterLC']!=161]

        #Excluding Russel
        pf = pf.loc[pf['MasterLC']!=265]
        
        #Excluding Diablo
        pf = pf.loc[pf['MasterLC']!=590]
        
        #Excluding SCT Performance
        pf = pf.loc[pf['MasterLC']!=837]
        
        #Removing the Holley Carburetor Repair Kit and Magnaflow Exhause System Kits
        eBay_Violation_skus = pd.read_csv('EBay Violation SKUs.csv', low_memory = False)        
        pf = pf.merge(eBay_Violation_skus, how='left', left_on='MasterSKU', right_on='SKU')
        pf = pf.loc[pf['SKU'].isnull()]
        pf.drop(columns = 'SKU', inplace = True)

        

    # Filter price file if APF or Mecka
    if channel['name'] == 'AP Fusion':
        apf_csskus = pd.read_csv('apf-skus.csv')['cssku']
        apf_csskus = apf_csskus.drop_duplicates()
        pf = pf[pf['CS-SKU-NP'].str[1:].isin(apf_csskus)]
    if channel['name'] == 'Mecka':
        pf = pf[(pf['WD']=='D') & (pf['item_cost'] >= 30)]
    # FIlter BS Walmart 
    if channel['name'] == 'BS Walmart':
        pf.drop(pf.loc[pf['WD'] == 'Y'].index, inplace=True)
        ##pf.drop(pf.loc[pf['WD'] == 'P'].index, inplace=True)
        pf.drop(pf.loc[pf['WD'] == 'J'].index, inplace=True)
        pf.drop(pf.loc[pf['WD'] == 'C'].index, inplace=True)
        pf.drop(pf.loc[pf['WD'] == '8'].index, inplace=True)        


    # # Calculate markups
    target_profit = channel['target_profit'] if 'target_profit' in channel else DEFAULT_TARGET_PROFIT
    channel_fees = channel['channel_transaction_fee'] + CS_TRANSACTION_FEE

    pf['our_cost'] = np.nan
    pf['our_markup'] = np.nan

    bfilter = pf['WD']=='C'
    pf.loc[bfilter, 'our_cost'] = pf.loc[bfilter, 'item_cost']

    # Handling Brock
    #+ $6 for any item costing < $50              
    pf.loc[bfilter & (pf['item_cost']<=50), 'our_cost' ] = pf.loc[bfilter & (pf['item_cost']<=50), 'item_cost' ] + 4
    # # 10% marup for all items, as buffer against no-return policy
    pf.loc[bfilter, 'our_cost' ] = pf.loc[bfilter, 'our_cost' ] * 1.1
    # calculate markup using same formula but with our updated cost value    
    pf.loc[bfilter, 'our_markup'] = pf.loc[bfilter, 'our_cost'] / (( 1+target_profit) * (pf.loc[bfilter, 'our_cost'] + pf.loc[bfilter, 'Shipping'])/ (1-channel_fees) - pf.loc[bfilter, 'Shipping'])
    # then adjust to find what the equivalent markup would be (to get to same final price) with the original cost value
    pf.loc[bfilter, 'Markup'] = pf.loc[bfilter, 'item_cost'] * pf.loc[bfilter, 'our_markup'] / pf.loc[bfilter, 'our_cost']
    pf.drop(columns=['our_cost', 'our_markup'], inplace=True)

    # handle Jante, certain skus specially
    if ( channel['name'] == 'PS Ebay'):
        f_1367 = pf['CS-SKU-NP'].str[:4]=='1367'
        pf.loc[f_1367, 'Markup'] = pf.loc[f_1367, 'item_cost'] / ( (1 + target_profit) * (pf.loc[f_1367, 'item_cost'] ) / (1 - channel_fees))

        # Added 1% more to channel fees  ---------------- this will never execute due to logical error - it will only execute if above condition is false which will never happen because 367 is the only line Jante has
        f_jante_n1367 = (pf['CS-SKU-NP'].str[:4]!='1367') & (pf['CS-SKU-NP'].str[0]=='1')
        pf.loc[f_jante_n1367, 'Markup'] = pf.loc[f_jante_n1367, 'item_cost'] / ( (1 + target_profit) * (pf.loc[f_jante_n1367, 'item_cost']) / (1 - (channel_fees + 0.01)))


    # Dorman increase profit
    f_dorman = pf['CS-SKU-NP'].str[0]=='D'   #Defining Filter for Dorman
    
    pf.loc[f_dorman, 'Markup']  = pf.loc[f_dorman, 'item_cost'] / ( (1 + 0.06) * (pf.loc[f_dorman, 'item_cost'] ) / (1 - channel_fees) )
    pf.loc[f_dorman, 'ShipMkup']  = pf.loc[f_dorman, 'Shipping'] / ( (1 + 0.06) * (pf.loc[f_dorman, 'Shipping'] ) / (1 - channel_fees) )
    
    
    g_300 = pf['item_cost'] >= 300
    pf.loc[f_dorman & g_300, 'Markup'] = pf.loc[f_dorman & g_300, 'item_cost'] / ( (1 + 0.0575) * (pf.loc[f_dorman & g_300, 'item_cost']) / (1 - channel_fees) )
    pf.loc[f_dorman & g_300, 'ShipMkup'] = pf.loc[f_dorman & g_300, 'Shipping'] / ( (1 + 0.0575) * (pf.loc[f_dorman & g_300, 'Shipping']) / (1 - channel_fees) )
    g_500 = pf['item_cost'] >= 500
    pf.loc[f_dorman & g_500, 'Markup'] = pf.loc[f_dorman & g_500, 'item_cost'] / ( (1 + 0.055) * (pf.loc[f_dorman & g_500, 'item_cost'] ) / (1 - channel_fees) )
    pf.loc[f_dorman & g_500, 'ShipMkup'] = pf.loc[f_dorman & g_500, 'Shipping'] / ( (1 + 0.055) * (pf.loc[f_dorman & g_500, 'Shipping'] ) / (1 - channel_fees) )
    g_1000 = pf['item_cost'] >= 1000
    pf.loc[f_dorman & g_1000, 'Markup'] = pf.loc[f_dorman & g_1000, 'item_cost'] / ( (1 + 0.05) * (pf.loc[f_dorman & g_1000, 'item_cost']) / (1 - channel_fees) )
    pf.loc[f_dorman & g_1000, 'ShipMkup'] = pf.loc[f_dorman & g_1000, 'Shipping'] / ( (1 + 0.05) * (pf.loc[f_dorman & g_1000, 'Shipping']) / (1 - channel_fees) )
    
    # base case, all other skus
    f_rem = pf['Markup'].isna()   #Defining Filter for remaining Data

    pf.loc[f_rem, 'Markup'] = pf.loc[f_rem, 'item_cost'] / ( (1 + target_profit) * (pf.loc[f_rem, 'item_cost']) / (1 - channel_fees) )
    pf.loc[f_rem, 'ShipMkup'] = pf.loc[f_rem, 'Shipping'] / ( (1 + target_profit) * (pf.loc[f_rem, 'Shipping']) / (1 - channel_fees) )
    

    # Format price file.
    if channel['name'] in CatSKU_CHANNELS:
        pf['CatSKU'] = (~pf['CS-SKU-NP'].str[1:].isin(noncat_skus)).map(lambda x: 'Y' if x else 'N')
        pf.loc[pf['CatSKU']=='Y', 'CS-SKU-NP'] = pf.loc[pf['CatSKU']=='Y', 'CS-SKU-NP-CatSKU']
    else:
        pf['CatSKU'] = 'N'
        

    # Clean up
    pf.loc[pf['Shipping']==0, 'ShipMkup'] = 1
    # pf.loc[pf['MinPrice'] < 1,'MinPrice'] = 1
    # pf.loc[pf['Markup'] < .1,'Markup'] = .1
    # pf.loc[pf['Markup'] > 1,'Markup'] = 1
    # pf['Markup'] = pf['Markup'].round(3)
    # pf['Shipping'] = pf['Shipping'].round(2)
    # pf.loc[pf['Shipping'].lt(0), 'Shipping'] = 0    
    pf['total_cost'] = pf['item_cost'] + pf['Shipping'] # hoping this will fix most examples of Dorman going thru PA    
    #pf['total_cost'] = (pf['item_cost'] * pf['PackQty']) / pf['Markup'] + pf['Shipping']/pf['ShipMkup']   #Computing the Final Total cost - For Future Use
    

    # give dorman direct preference over dorman skus from other warehouses
    dorman_prefs = pf[(pf['WD']=='D') & (pf['Qty'] > 0)]['MasterSKU'].copy().tolist()
    pf = pf[~((pf['WD'] != 'D') & (pf['MasterSKU'].isin(dorman_prefs)))]

    
    pf = pf.sort_values(['MasterSKU','Qty','total_cost'], ascending=[True,False,True])\
           .drop_duplicates(subset=['MasterSKU'], keep='first')
    
    #Alternate
    #pf = pf.sort_values(['MasterSKU','total_cost', 'Qty'], ascending=[True,True,False])\
    #        .drop_duplicates(subset=['MasterSKU'], keep='first')


    # Return the Price File
    return pf

In [45]:
for c in CHANNELS:
    t1 = time.time()
    pf = process_channel(c)   #Processing the channel and getting Price File
    
    pf = pf[PRICE_FILE_COLUMNS]   #Only taking the columns needed for the Price File
    
    # Write price file to Disk
    pf.to_csv(f"{PRICE_FILE_LOCATION}/{c['name']}.csv", index=False)
    
    t2 = time.time()
    print (c, round(t2-t1,3))
    
end_time = time.time()

print ('Total Time Taken : ', end_time - start_time, ' seconds')

{'name': 'AP Fusion', 'channel_transaction_fee': 0.08, 'target_profit': 0.09} 3.105
{'name': 'PS Amazon', 'channel_transaction_fee': 0.119} 20.097
{'name': 'PS Walmart', 'channel_transaction_fee': 0.125} 22.69
{'name': 'PS Ebay', 'channel_transaction_fee': 0.17} 28.454
{'name': 'BS Amazon', 'channel_transaction_fee': 0.12} 22.196
{'name': 'BS Walmart', 'channel_transaction_fee': 0.12} 23.499
{'name': 'BS Ebay', 'channel_transaction_fee': 0.12} 27.201
{'name': 'Mecka', 'channel_transaction_fee': 0.12} 1.197
Total Time Taken :  230.53219628334045  seconds


{'name': 'PS Amazon', 'channel_transaction_fee': 0.119}

In [46]:
c = CHANNELS[1]

In [63]:
old = process_channel(c)
new = process_channel_new(c)

In [95]:
old['fprice'] = (old['MinPrice'] * old['PackQty'])/old['Markup'] + old['Shipping']/1.0

new['fprice'] = (new['MinPrice'] * new['PackQty'])/new['Markup'] + new['Shipping']/new['ShipMkup']

In [96]:
cmp = old.merge(new, on='CS-SKU-NP')
cmp['WD'] = cmp['CS-SKU-NP'].str[0]

In [97]:
tp = DEFAULT_TARGET_PROFIT
channel_fee = c['channel_transaction_fee'] + CS_TRANSACTION_FEE

In [112]:
pa = cmp.query("WD == 'P' and PackQty_x==1 ")[['WD', 'PackQty_x', 'CS-SKU-NP', 'MinPrice_x', 'MinPrice_y', 'Markup_x', 'Markup_y', 'Shipping_x', 'Shipping_y', 'ShipMkup_x', 'ShipMkup_y', 'fprice_x', 'fprice_y']]

In [113]:
pa['TP'] = (1 - channel_fee) * (1/pa['ShipMkup_x'] - 1) + 0.05

pa['ShipMkup_1']  = pa['Shipping_y'] / ( (1 + pa['TP']) * (pa['Shipping_y'] ) / (1 - channel_fee) )

pa['fprice_0'] = (pa['MinPrice_x'] * pa['PackQty_x'])/pa['Markup_x'] + pa['Shipping_x']/pa['ShipMkup_x']

pa['fprice_1'] = (pa['MinPrice_y'] * pa['PackQty_x'])/pa['Markup_y'] + pa['Shipping_y']/pa['ShipMkup_1']

In [119]:
pa.query(" abs(fprice_1 - fprice_0) < 0.01 ")

,WD,PackQty_x,CS-SKU-NP,MinPrice_x,MinPrice_y,Markup_x,Markup_y,Shipping_x,Shipping_y,ShipMkup_x,ShipMkup_y,fprice_x,fprice_y,TP,ShipMkup_1,fprice_0,fprice_1
461,P,1.0,P102|01003,2.65,2.65,0.593388,0.829429,6.180000,6.180000,0.893,0.829429,10.645884,10.645884,0.154352,0.754449,11.386376,11.386376
462,P,1.0,P102|01003BP,2.79,2.79,0.478432,0.829429,12.000000,12.000000,0.893,0.829429,17.831554,17.831554,0.154352,0.754449,19.269404,19.269404
463,P,1.0,P102|01004,1.85,1.85,0.553733,0.829429,5.400000,5.400000,0.893,0.829429,8.740958,8.740958,0.154352,0.754449,9.387990,9.387990
464,P,1.0,P102|01004BP,3.56,3.56,0.679903,0.829429,4.590000,4.590000,0.893,0.829429,9.826042,9.826042,0.154352,0.754449,10.376020,10.376020
465,P,1.0,P102|01005,2.94,2.94,0.631563,0.829429,5.400000,5.400000,0.893,0.829429,10.055115,10.055115,0.154352,0.754449,10.702148,10.702148
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1225624,P,1.0,P981|920037,74.17,74.17,0.813318,0.829429,8.613204,8.613204,0.893,0.829429,99.807514,99.807514,0.154352,0.754449,100.839555,100.839555
1225625,P,1.0,P981|920137,83.43,83.43,0.815075,0.829429,8.613204,8.613204,0.893,0.829429,110.971826,110.971826,0.154352,0.754449,112.003868,112.003868
1225626,P,1.0,P981|920138,83.43,83.43,0.815075,0.829429,8.613204,8.613204,0.893,0.829429,110.971826,110.971826,0.154352,0.754449,112.003868,112.003868
1225627,P,1.0,P981|930288,14.23,14.23,0.751809,0.829429,8.613204,8.613204,0.893,0.829429,27.540893,27.540893,0.154352,0.754449,28.572934,28.572934


In [121]:
pa.to_excel('tmp PA working.xlsx')

In [120]:
pa.groupby('TP').size()

TP
0.154352    870775
dtype: int64

870775

Dorman has PackQty issue

In [56]:
cmp.query(" (fprice_x - fprice_y)<0.1 and WD=='D' and PackQty_x>1")[['WD', 'PackQty_x', 'CS-SKU-NP', 'MinPrice_x', 'MinPrice_y', 'Markup_x', 'Markup_y', 'Shipping_x', 'Shipping_y', 'ShipMkup_x', 'ShipMkup_y', 'fprice_x', 'fprice_y']]

,WD,PackQty_x,CS-SKU-NP,MinPrice_x,MinPrice_y,Markup_x,Markup_y,Shipping_x,Shipping_y,ShipMkup_x,ShipMkup_y,fprice_x,fprice_y
981037,D,2.0,D591|523280,72.47,72.47,0.821604,0.821604,0.0,0.0,1,1.0,176.411069,176.411069
981038,D,2.0,D591|523281,72.47,72.47,0.821604,0.821604,0.0,0.0,1,1.0,176.411069,176.411069
995371,D,2.0,D591|615128,63.08,63.08,0.821604,0.821604,0.0,0.0,1,1.0,153.553336,153.553336


In [60]:
cmp.query(" abs(fprice_x - fprice_y)>0.1 and WD=='N'")[['WD', 'PackQty_x', 'CS-SKU-NP', 'MinPrice_x', 'MinPrice_y', 'Markup_x', 'Markup_y', 'Shipping_x', 'Shipping_y', 'ShipMkup_x', 'ShipMkup_y', 'fprice_x', 'fprice_y']]

,WD,PackQty_x,CS-SKU-NP,MinPrice_x,MinPrice_y,Markup_x,Markup_y,Shipping_x,Shipping_y,ShipMkup_x,ShipMkup_y,fprice_x,fprice_y
13181,N,10.0,N111|23609,1.70,1.70,0.376323,0.829429,12.0,12.0,1,0.829429,57.173958,34.963831
14848,N,5.0,N111|420,0.64,0.64,0.197567,0.829429,12.0,12.0,1,0.829429,28.197038,18.325870
14849,N,5.0,N111|421,0.70,0.70,0.211369,0.829429,12.0,12.0,1,0.829429,28.558732,18.687565
14854,N,5.0,N111|425RR,0.82,0.82,0.237239,0.829429,12.0,12.0,1,0.829429,29.282122,19.410954
14859,N,5.0,N111|428RR,0.86,0.86,0.245388,0.829429,12.0,12.0,1,0.829429,29.523252,19.652084
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1097312,N,2.0,N643|JBR999XPR,92.59,92.59,0.811489,0.829429,12.0,12.0,1,0.829429,240.197726,237.729935
1123895,N,6.0,N650|W2070,7.52,7.52,0.651970,0.829429,12.0,12.0,1,0.829429,81.205649,68.866690
1124449,N,6.0,N650|W32110,7.72,7.72,0.655604,0.829429,12.0,12.0,1,0.829429,82.652429,70.313469
1126236,N,6.0,N650|W9112,10.70,10.70,0.696241,0.829429,12.0,12.0,1,0.829429,104.209439,91.870479


In [193]:
cmp.query(" abs(fprice_x - fprice_y)<0.1 and WD=='K' ")[['WD', 'PackQty_x', 'CS-SKU-NP', 'MinPrice_x', 'MinPrice_y', 'Markup_x', 'Markup_y', 'Shipping_x', 'Shipping_y', 'ShipMkup_x', 'ShipMkup_y', 'fprice_x', 'fprice_y']]

,WD,PackQty_x,CS-SKU-NP,MinPrice_x,MinPrice_y,Markup_x,Markup_y,Shipping_x,Shipping_y,ShipMkup_x,ShipMkup_y,fprice_x,fprice_y
50,K,1.0,K101|037044,31.54,31.54,0.782857,0.829429,11.0,11.0,1,0.829429,51.288322,51.288322
51,K,1.0,K101|038510,113.16,113.16,0.815900,0.829429,11.0,11.0,1,0.829429,149.693421,149.693421
56,K,1.0,K101|301311,111.70,111.70,0.815726,0.829429,11.0,11.0,1,0.829429,147.933173,147.933173
58,K,1.0,K101|301391,99.20,99.20,0.814032,0.829429,11.0,11.0,1,0.829429,132.862556,132.862556
70,K,1.0,K101|301691,111.70,111.70,0.815726,0.829429,11.0,11.0,1,0.829429,147.933173,147.933173
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224351,K,1.0,K980|TGH65537,14.80,14.80,0.736108,0.829429,11.0,11.0,1,0.829429,31.105753,31.105753
1224352,K,1.0,K980|TGH65538,2.21,2.21,0.448583,0.829429,11.0,11.0,1,0.829429,15.926628,15.926628
1224353,K,1.0,K980|TGH66100,59.47,59.47,0.804060,0.829429,11.0,11.0,1,0.829429,84.962108,84.962108
1224354,K,1.0,K980|WT500,67.72,67.72,0.807068,0.829429,11.0,11.0,1,0.829429,94.908715,94.908715


In [62]:
cmp.query(" abs(fprice_x - fprice_y)>0.1 and WD=='P' ")[['WD', 'PackQty_x', 'CS-SKU-NP', 'MinPrice_x', 'MinPrice_y', 'Markup_x', 'Markup_y', 'Shipping_x', 'Shipping_y', 'ShipMkup_x', 'ShipMkup_y', 'fprice_x', 'fprice_y']].to_excel('pa_tmp.xlsx')

In [201]:
cmp.query(" abs(fprice_x - fprice_y)<0.1 and WD=='J' ")[['WD', 'PackQty_x', 'CS-SKU-NP', 'MinPrice_x', 'MinPrice_y', 'Markup_x', 'Markup_y', 'Shipping_x', 'Shipping_y', 'ShipMkup_x', 'ShipMkup_y', 'fprice_x', 'fprice_y']]

,WD,PackQty_x,CS-SKU-NP,MinPrice_x,MinPrice_y,Markup_x,Markup_y,Shipping_x,Shipping_y,ShipMkup_x,ShipMkup_y,fprice_x,fprice_y
514097,J,1.0,J366|016-0050,4.61,4.61,0.567273,0.829429,12.49,12.49,1,0.829429,20.616604,20.616604
514098,J,1.0,J366|016-0053,3.32,3.32,0.505226,0.829429,12.49,12.49,1,0.829429,19.061316,19.061316
514099,J,1.0,J366|016-0088,1.64,1.64,0.360771,0.829429,12.49,12.49,1,0.829429,17.035825,17.035825
514100,J,1.0,J366|016-0116,1.20,1.20,0.308329,0.829429,11.89,11.89,1,0.829429,15.781950,15.781950
514101,J,1.0,J366|016-0138,3.25,3.25,0.501008,0.829429,12.49,12.49,1,0.829429,18.976920,18.976920
...,...,...,...,...,...,...,...,...,...,...,...,...,...
594811,J,1.0,J366|YUKYYC5189950,126.10,126.10,0.811117,0.829429,16.69,16.69,1,0.829429,172.154668,172.154668
594812,J,1.0,J366|YUKYYD44133026U,88.01,88.01,0.803440,0.829429,16.69,16.69,1,0.829429,126.231485,126.231485
594813,J,1.0,J366|YUKYYD60135029S,128.64,128.64,0.809371,0.829429,18.69,18.69,1,0.829429,177.628316,177.628316
594814,J,1.0,J366|YUKYYF100603,151.43,151.43,0.813224,0.829429,17.69,17.69,1,0.829429,203.899414,203.899414


In [214]:
cmp.query(" abs(fprice_x - fprice_y)<0.1 and WD=='1' ")[['WD', 'PackQty_x', 'CS-SKU-NP', 'MinPrice_x', 'MinPrice_y', 'Markup_x', 'Markup_y', 'Shipping_x', 'Shipping_y', 'ShipMkup_x', 'ShipMkup_y', 'fprice_x', 'fprice_y']]

,WD,PackQty_x,CS-SKU-NP,MinPrice_x,MinPrice_y,Markup_x,Markup_y,Shipping_x,Shipping_y,ShipMkup_x,ShipMkup_y,fprice_x,fprice_y
594816,1,1.0,1367|AFM03995U45N,224.88,224.88,0.829429,0.829429,0.0,0.0,1,1.0,271.126421,271.126421
594817,1,1.0,1367|AFM03999U45N,251.36,251.36,0.829429,0.829429,0.0,0.0,1,1.0,303.052015,303.052015
594818,1,1.0,1367|AFM05300U45N,256.90,256.90,0.829429,0.829429,0.0,0.0,1,1.0,309.731312,309.731312
594819,1,1.0,1367|AFM05657U45N,244.86,244.86,0.829429,0.829429,0.0,0.0,1,1.0,295.215295,295.215295
594820,1,1.0,1367|AFM05879U45N,201.98,201.98,0.829429,0.829429,0.0,0.0,1,1.0,243.517051,243.517051
...,...,...,...,...,...,...,...,...,...,...,...,...,...
600986,1,1.0,1367|WCC85027U68OET,13.32,13.32,0.829429,0.829429,0.0,0.0,1,1.0,16.059249,16.059249
600987,1,1.0,1367|WCC85297U45OET,17.70,17.70,0.829429,0.829429,0.0,0.0,1,1.0,21.339993,21.339993
600988,1,1.0,1367|WCC85390U45OET,13.32,13.32,0.829429,0.829429,0.0,0.0,1,1.0,16.059249,16.059249
600989,1,1.0,1367|WCC85509U45NOE,46.22,46.22,0.829429,0.829429,0.0,0.0,1,1.0,55.725112,55.725112


In [216]:
pw.query("WD == 'D' and PackQty>1").to_excel('Dorman Packs tmp.xlsx')